In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mlops/.ivy2/cache
The jars for the packages stored in: /home/mlops/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4db07b5-c37b-4e6c-aac8-afca3eb0aec7;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 367ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      def

In [3]:
ticketmaster = spark.read.format('delta').load(
    '/home/mlops/project/DeltaLake/platinum_data/tfx_genre_as_label')
ticketmaster.to_pandas_on_spark()

,spotify_followers,spotify_genres,spotify_href,spotify_id,spotify_name,spotify_popularity,spotify_type,spotify_uri,ticketmaster_name,ticketmaster_id,ticketmaster_locale,ticketmaster_distance,ticketmaster_units,ticketmaster_price_min,ticketmaster_price_max,spotify_url,spotify_image_url,ticketmaster_date,ticketmaster_venue_name,ticketmaster_venue_id,ticketmaster_venue_url,ticketmaster_venue_postal_code,ticketmaster_venue_timezone,ticketmaster_venue_city,ticketmaster_venue_country,ticketmaster_venue_address,ticketmaster_url,ticketmaster_artist_locale,ticketmaster_venue_latitude,ticketmaster_venue_longitude,spotify_genres_9,spotify_genres_18,spotify_genres_1
0,66848,"[album rock, mellow gold, new wave pop, philly...",https://api.spotify.com/v1/artists/7uhvDINTTiD...,7uhvDINTTiD0XBrP9fquN1,The Hooters,56.0,artist,spotify:artist:7uhvDINTTiD0XBrP9fquN1,The Hooters,Z698xZC2Z17uPao,en-us,67.49,MILES,35.00,35.00,https://open.spotify.com/artist/7uhvDINTTiD0XB...,https://i.scdn.co/image/ab6761610000e5eb15896b...,2022-06-28,Musiktheater REX,Z698xZC2Za7r9,https://www.ticketmaster.de/venue/musiktheater...,64625,Europe/Berlin,Bensheim,Germany,Fabrikstraße 10,http://www.ticketmaster.de/artist/918988,en-us,49.680740,8.619640,"[Rock and Heavy Metal, Pop]","[Rock, Pop]",Rock and Heavy Metal
1,760031,"[francoton, pop urbaine]",https://api.spotify.com/v1/artists/7gU9VyFRN3J...,7gU9VyFRN3JWPJ5oHOil60,Tayc,76.0,artist,spotify:artist:7gU9VyFRN3JWPJ5oHOil60,TAYC,rZ7SnyZ1Ad700d,fr-fr,67.72,MILES,26.00,26.00,https://open.spotify.com/artist/7gU9VyFRN3JWPJ...,https://i.scdn.co/image/ab6761610000e5ebce9a6f...,2021-12-16,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/1044372,en-us,48.576090,7.729777,[Pop],[Pop],Pop
2,876137,"[electro swing, nu jazz]",https://api.spotify.com/v1/artists/37J1PlAkhRK...,37J1PlAkhRK7yrZUtqaUpQ,Caravan Palace,66.0,artist,spotify:artist:37J1PlAkhRK7yrZUtqaUpQ,CARAVAN PALACE,rZ7SnyZ1Adk7qv,fr-fr,67.72,MILES,34.20,34.20,https://open.spotify.com/artist/37J1PlAkhRK7yr...,https://i.scdn.co/image/ab6761610000e5ebdedbe7...,2021-12-18,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/141183,en-us,48.576090,7.729777,"[Electronical music, Rhythm music]","[Jazz, Dance Music, Techno and House]",Electronical music
3,139871,[cali rap],https://api.spotify.com/v1/artists/1grN0519h2z...,1grN0519h2zYqpRtYbDZAl,Larry June,67.0,artist,spotify:artist:1grN0519h2zYqpRtYbDZAl,LARRY,rZ7SnyZ1AduOGE,fr-fr,67.72,MILES,26.00,26.00,https://open.spotify.com/artist/1grN0519h2zYqp...,https://i.scdn.co/image/ab6761610000e5eb1c0b98...,2022-01-15,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",None,None,48.576090,7.729777,[Rap and Hip-Hop],[Rap and Hip-Hop],Rap and Hip-Hop
4,40482,"[electro swing, nu jazz]",https://api.spotify.com/v1/artists/6jX8VHUJomY...,6jX8VHUJomYSfi5Hobdmmn,Lyre Le Temps,47.0,artist,spotify:artist:6jX8VHUJomYSfi5Hobdmmn,LYRE LE TEMPS,rZ7SnyZ1AdGyqo,fr-fr,67.72,MILES,17.00,17.00,https://open.spotify.com/artist/6jX8VHUJomYSfi...,https://i.scdn.co/image/ab6761610000e5eb78c466...,2022-01-21,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",None,None,48.576090,7.729777,"[Electronical music, Rhythm music]","[Jazz, Dance Music, Techno and House]",Electronical music
5,5707,[neo metal],https://api.spotify.com/v1/artists/0gPevmj7hYa...,0gPevmj7hYafkYuMEGCFCh,Psykup,22.0,artist,spotify:artist:0gPevmj7hYafkYuMEGCFCh,PSYKUP,rZ7SnyZ1AdAYPb,fr-fr,67.72,MILES,14.00,14.00,https://open.spotify.com/artist/0gPevmj7hYafkY...,https://i.scdn.co/image/ab6761610000e5ebc9cb34...,2022-01-22,LA LAITERIE - Club,rZFSnyZFAAa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe

In [4]:
rdy_to_serve = ticketmaster

rdy_to_serve = rdy_to_serve.select('spotify_genres_1', 'spotify_followers', 'spotify_popularity')

rdy_to_serve.to_pandas_on_spark().head()

,spotify_genres_1,spotify_followers,spotify_popularity
0,Rock and Heavy Metal,66848,56.0
1,Pop,760031,76.0
2,Electronical music,876137,66.0
3,Rap and Hip-Hop,139871,67.0
4,Electronical music,40482,47.0


In [5]:
rdy_to_serve2 = rdy_to_serve

rdy_to_serve2 = rdy_to_serve2.withColumn('spotify_followers', rdy_to_serve2.spotify_followers.cast('float'))

rdy_to_serve2.to_pandas_on_spark().head()

,spotify_genres_1,spotify_followers,spotify_popularity
0,Rock and Heavy Metal,66848.0,56.0
1,Pop,760031.0,76.0
2,Electronical music,876137.0,66.0
3,Rap and Hip-Hop,139871.0,67.0
4,Electronical music,40482.0,47.0


In [6]:
genre_map = spark.read.format('delta').load('/home/mlops/project/DeltaLake/platinum_data/genre_map_indexed')
genre_map.to_pandas_on_spark().head()

,genres,main_genre_18,main_genre_9,main_genre_9_index,main_genre_18_index,genres_index
0,pop,Pop,Pop,1,0,4115
1,dance pop,Pop,Pop,1,0,1276
2,rap,Rap and Hip-Hop,Rap and Hip-Hop,5,5,4327
3,rock,Rock,Rock and Heavy Metal,0,2,4469
4,latin,Latin American music,Latin American music,8,10,3049


In [20]:
rdy_to_serve3 = rdy_to_serve2

genre_map_prepared = genre_map.select('main_genre_9', 'main_genre_9_index')\
    .withColumnRenamed('main_genre_9', 'spotify_genres_1')\
    .withColumnRenamed('main_genre_9_index', 'spotify_genres_1_index').distinct()

rdy_to_serve3 = rdy_to_serve3.join(genre_map_prepared, 'spotify_genres_1').drop('spotify_genres_1')

rdy_to_serve3.to_pandas_on_spark()

,spotify_followers,spotify_popularity,spotify_genres_1_index
0,61507.0,61.0,1
1,728400.0,56.0,1
2,55108460.0,98.0,1
3,87307.0,57.0,1
4,339459.0,63.0,1
5,40436352.0,97.0,1
6,952797.0,66.0,1
7,55060960.0,93.0,1
8,5606785.0,80.0,1
9,5606785.0,80.0,1


In [21]:
rdy_to_serve3.write.format("delta").mode("overwrite").save(
    "/home/mlops/project/DeltaLake/platinum_data/ticketmaster_rdy_to_serve")

In [22]:

ticketmaster = spark.read.format('delta').load(
    '/home/mlops/project/DeltaLake/platinum_data/ticketmaster_rdy_to_serve')
ticketmaster.to_pandas_on_spark()

,spotify_followers,spotify_popularity,spotify_genres_1_index
0,61507.0,61.0,1
1,728400.0,56.0,1
2,55108460.0,98.0,1
3,87307.0,57.0,1
4,339459.0,63.0,1
5,40436352.0,97.0,1
6,952797.0,66.0,1
7,55060960.0,93.0,1
8,5606785.0,80.0,1
9,5606785.0,80.0,1
